In [1]:
import sys
import numpy as np
import pandas as pd
from more_itertools import locate
import os

import json
import pickle


In [97]:
# 获取当前目录路径
current_dir = os.getcwd()

# 定义子目录名
subdir_name = 'Output_treads'
# subdir_name1 = 'Output_pelicun'
# subdir_name2 = 'Output_preprocessing'

# 创建子目录
subdir_path = os.path.join(current_dir, subdir_name)
# subdir_pelicun = os.path.join(current_dir, subdir_name1)
# subdir_preprocessing = os.path.join(current_dir, subdir_name2)


In [3]:
# 构建pickle文件的完整路径 导入RC_results
# pickle_file_path = os.path.join(subdir_treads, 'RC_results.pkl')
pickle_file_path = 'Output_treads/RC_results.pkl'

# 从pickle文件中加载字典数据
with open(pickle_file_path, "rb") as file:
    RC_results = pickle.load(file)

In [4]:
RCtable_input = 'Repair_Class_Table_R.csv'
DMG_input = 'Output_preprocessing/DMG_sample_R.csv' #pelicun output 经过前处理，去掉了倒塌和不可修复的情况，和无损伤的样本
DV_rec_time_input = 'Output_preprocessing/loss_sample_time_DMG.csv'
input_params_json = 'Output_preprocessing/input_parameters_R.json'
DL_summary_input = 'Output_pelicun/DL_summary.csv' #pelicun output

RC = RC_results["RC"]
RCmax_RS = np.squeeze(RC_results["RCmax_RS"])

output_path = subdir_path

In [5]:
with open(input_params_json) as f:
    input_param = json.load(f)

In [6]:
if sum(input_param['repair_phases'])!=len(RCmax_RS.T)/7:  #7为维修顺序数量（RS1-RS7）
    sys.exit("Sum of repair phases is not equal to the number of stories")

In [7]:
#repair time calculation
n_elev = input_param['elevator_quantity']
rep_phases = input_param['repair_phases']
fl_area = input_param['floor_area']
max_num_workers = input_param['max_number_workers']

In [8]:
RC_table = pd.read_csv(RCtable_input) 
DMG = pd.read_csv(DMG_input, low_memory=False, header=None)

DV_rec_time = pd.read_csv(DV_rec_time_input, low_memory=False, header=None)
rec_time = DV_rec_time.loc[4:,0:]
DMG_all = DMG.loc[4:, 1:]
RC_data = RC.iloc[4:,1:]
FG_all = RC_table["Fragility"]
Qty_FEMA = RC_table['Qty unit FEMA P-58']
Wrk_DMG = RC_table['Workers per damaged Qty']


FG = DMG.loc[0, 1:] #fragility groups 
n_PG = len(FG)  #number of performance groups 不同EDP，不同方向，不同楼层
DL_summary=pd.read_csv(DL_summary_input)

#determine indices for collpase, irreparable, and repairable realizations
cases_collapse = DL_summary["collapses/collapsed"]
indx_collapse = cases_collapse[cases_collapse==1].index
cases_irreparable = DL_summary["reconstruction/irreparable"]
indx_irreparable = cases_irreparable[cases_irreparable==1].index
indx_all = DL_summary["#Num"]
indx_repairable =  indx_all.drop(indx_collapse.union(indx_irreparable))

for i in range(len(DMG_all)):
    if int(DMG.loc[i+4, 0]) in indx_irreparable:            
        DMG_all.iloc[i,:]=np.nan
DMG_repairable = DMG_all.dropna()
DMG_repairable = DMG_repairable.astype('float')
DMG_repairable = np.array(DMG_repairable)

RT_RC2 = np.zeros((len(indx_repairable),n_PG))  #repair times for functional recovery 
DMG_RC2 = np.zeros((len(indx_repairable),n_PG))


In [ ]:
DMG

In [ ]:
FG

In [ ]:
RT_RC2.shape


In [9]:
#determine the number of damaged components and repair times for each recovery state
for i in range(len(indx_repairable)):
    for j in range(n_PG):
        if DMG_repairable[i,j] != 0 and float(RC_data.iloc[i,j]) >= 2:
            RT_RC2[i,j] = float(rec_time.iloc[i,j])
            DMG_RC2[i,j] = DMG_repairable[i,j]
            


In [10]:
#create matrix of FEMA P-58 coefficients for fragilities
Qty_norm=[]
Wrk_norm=[]
for i in range(n_PG):
    indx_FG = FG_all[FG_all==FG[i+1]].index
    Qty_norm.append(Qty_FEMA[indx_FG])
    Wrk_norm.append(Wrk_DMG[indx_FG])
Qty_norm = np.stack( Qty_norm, axis=0)
Wrk_norm = np.stack( Wrk_norm, axis=0)
Qty_norm_mat = np.repeat(Qty_norm.T, len(indx_repairable), axis=0)
Wrk_norm_mat = np.repeat(Wrk_norm.T, len(indx_repairable), axis=0)

In [11]:
#number of damaged components matrix for each recovery state
N_DMG_RC2 = np.ceil(np.divide(DMG_RC2,Qty_norm_mat)) 


In [ ]:
N_DMG_RC2.shape

In [12]:
#edit the number of damaged FEMA P-58 elevator components (simultaneous damage states)
PG = DMG.loc[1, 1:]
story_FG = []
for i in range(n_PG):
    story_FG.append(int((PG[i+1])))

story = list(set(story_FG))
story.sort()

indx_PG1 = (PG[PG=='1'].index)-1
indx_FG1 = (FG[FG=='D.10.14.011'].index)-1 #易损性组只有构件类型有关
indx = indx_PG1.intersection(indx_FG1)
for k in range(len(indx_repairable)):
    maxx = np.max(N_DMG_RC2[k][indx], initial=0.0001)
    a = N_DMG_RC2[k][indx]
    a[a<maxx]=0
    N_DMG_RC2[k][indx] = (a*n_elev[0])/np.max([n_elev[0],np.sum(a)]) #电梯损伤单元个数
    

indx_PG1 = (PG[PG=='30'].index)-1
indx_FG1 = (FG[FG=='D.10.14.011'].index)-1
indx = indx_PG1.intersection(indx_FG1)
for k in range(len(indx_repairable)):
    maxx = np.max(N_DMG_RC2[k][indx], initial=0.0001)
    a = N_DMG_RC2[k][indx]
    a[a<maxx]=0
    N_DMG_RC2[k][indx] = (a*n_elev[1])/np.max([n_elev[1],np.sum(a)]) #电梯损伤单元个数
    
# indx = (FG[FG=='D1014.012'].index)-1
# for k in range(len(indx_repairable)):
#     maxx = np.max(N_DMG_RC2[k][indx], initial=0.0001)
#     a = N_DMG_RC2[k][indx]
#     a[a<maxx]=0
#     N_DMG_RC2[k][indx] = (a*n_elev)/np.max([n_elev,np.sum(a)])
    
# indx = (FG[FG=='D1014.021'].index)-1
# for k in range(len(indx_repairable)):
#     maxx = np.max(N_DMG_RC2[k][indx], initial=0.0001)
#     a = N_DMG_RC2[k][indx]
#     a[a<maxx]=0
#     N_DMG_RC2[k][indx] = (a*n_elev)/np.max([n_elev,np.sum(a)])

# indx = (FG[FG=='D1014.022'].index)-1
# for k in range(len(indx_repairable)):
#     maxx = np.max(N_DMG_RC2[k][indx], initial=0.0001)
#     a = N_DMG_RC2[k][indx]
#     a[a<maxx]=0
#     N_DMG_RC2[k][indx] = (a*n_elev)/np.max([n_elev,np.sum(a)])     

In [15]:
#an original matrix for number of workers is generated using the values defined in the repair class table
N_Wrk_RC2 = (np.multiply(N_DMG_RC2,Wrk_norm_mat)) #number of workers matrix



In [20]:
#%% repair time calculation per sequence per story

FG_all = RC_table["Fragility"]
RS_all = RC_table["Repair Sequence"]

RS_FG =[] #每个易损性组对应的维修顺序

for i in range(n_PG):    
    indx_FG = FG_all[FG_all==FG[i+1]].index
    RS_FG.append(RS_all[indx_FG[0]])  #每个易损性对应的维修顺序(RS)

RS = list(set(RS_FG))
RS.sort()

header=[]
for i in range(len(story)):
    for j in range(len(RS)):
        header.append(str(story[i])+'_'+str(RS[j]))
header_ = np.vstack((header, [""]*len(header)))
col = np.append('St_RSeq', np.append('#Num', np.arange(0,len(indx_repairable),1))) #按照楼层和修复顺序分组

In [23]:
RT_RC2_RS=[]
N_Wrk_RC2_RS=[]

for i in range(len(indx_repairable)):
    RT_RC2_realization = RT_RC2[i,:]
    N_Wrk_RC2_realization = N_Wrk_RC2[i,:]
    RT_RC2_RS.append([])
    N_Wrk_RC2_RS.append([])

    for j in range(len(story)):
        for k in range(len(RS)):
            indx_ST = list(locate(story_FG, lambda x: x == story[j]))
            indx_RS = list(locate(RS_FG, lambda x: x == RS[k]))
            indx_intersect = list(set.intersection(set(indx_ST),set(indx_RS)))  #安装楼层和维修顺序分组
            RT_RC2_RS[i].append(np.sum(RT_RC2_realization[indx_intersect]))
            N_Wrk_RC2_RS[i].append(np.sum(N_Wrk_RC2_realization[indx_intersect]))
            


In [24]:
story[1]

2

In [25]:
#%% worker allocation limits
N_Wrk_RC2_RS_org = np.squeeze(N_Wrk_RC2_RS)
N_Wrk_RC2_RS_adj = np.squeeze(N_Wrk_RC2_RS)  #调整工人数量


In [ ]:
N_Wrk_RC2_RS.size


In [26]:
max_worker_fl = np.zeros((len(indx_repairable),len(story))) #工人数量限值
for i in range(len(indx_repairable)):
    for j in range(len(story)):
        if (float(np.max(RC.iloc[i+4,:])) >= 4):
            max_worker_fl[i,j] = (1/500)*fl_area[j]
        else: max_worker_fl[i,j] = (1/1000)*fl_area[j]

if (len(story) <= 5):
    max_worker_seq = max_num_workers
elif (len(story) > 5) and (len(story) <= 20):
    max_worker_seq = [2 * k for k in max_num_workers]
else:
    max_worker_seq = [3 * k for k in max_num_workers]

In [27]:
#worker limit per repair sequence based on the floor area
for i in range(len(indx_repairable)):
    for j in range(len(story)):
        N_Wrk_RC2_RS_adj[i,j*7] = min(N_Wrk_RC2_RS_org[i,j*7], max_worker_fl[i,j])
        
        N_Wrk_RC2_RS_adj[i,j*7+2] = min(N_Wrk_RC2_RS_org[i,j*7+2], max_worker_fl[i,j])
        
        N_Wrk_RC2_RS_adj[i,j*7+5] = min(N_Wrk_RC2_RS_org[i,j*7+5], max_worker_fl[i,j])
        
        N_Wrk_RC2_RS_adj[i,j*7+6] = min(N_Wrk_RC2_RS_org[i,j*7+6], max_worker_fl[i,j])
        
        summ2 =  N_Wrk_RC2_RS_org[i,j*7+1]+N_Wrk_RC2_RS_org[i,j*7+3]+N_Wrk_RC2_RS_org[i,j*7+4]
        
        if (summ2 > max_worker_fl[i,j]): #修复阶段2（RS2-RS4-RS5）的工人数量超限时，按比例分配
            N_Wrk_RC2_RS_adj[i,j*7+1] = np.round((N_Wrk_RC2_RS_org[i,j*7+1])*(max_worker_fl[i,j]/summ2))
            N_Wrk_RC2_RS_adj[i,j*7+3] = np.round((N_Wrk_RC2_RS_org[i,j*7+3])*(max_worker_fl[i,j]/summ2))
            N_Wrk_RC2_RS_adj[i,j*7+4] = np.round((N_Wrk_RC2_RS_org[i,j*7+4])*(max_worker_fl[i,j]/summ2))


In [28]:
#worker limit per repair sequence based on the numbers defined in Paul et al. (2018) table 1
N_Wrk_RC2_RS_adj2 = N_Wrk_RC2_RS_adj

for i in range(len(indx_repairable)):
    x=0
    for j in range(len(rep_phases)):
        for k in range(len(RS)):
            summ = sum(N_Wrk_RC2_RS_adj[i,k+x:len(RS)*rep_phases[j]+k+x:len(RS)]) 
            if (summ > max_worker_seq[k]):
                N_Wrk_RC2_RS_adj2[i,k+x:len(RS)*rep_phases[j]+k+x:len(RS)] = np.round(N_Wrk_RC2_RS_adj[i,k+x:len(RS)*rep_phases[j]+k+x:len(RS)] * (max_worker_seq[k]/summ))
        x = rep_phases[j]*7+x
        


In [29]:
#worker limit based on the total floor area defined in the REDi guidelines
bld_area_ft2 = sum(fl_area)
max_worker_tot1 = max(2.5*0.0001*bld_area_ft2 + 10, 20)
max_worker_tot = min(max_worker_tot1, 260)

N_Wrk_RC2_RS_adj3 = N_Wrk_RC2_RS_adj2

for i in range(len(indx_repairable)):
    y=0
    for j in range(len(rep_phases)):
        summ = sum(N_Wrk_RC2_RS_adj2[i,y:y+rep_phases[j]*7])
        if (summ > max_worker_tot):
            N_Wrk_RC2_RS_adj3[i,y:y+rep_phases[j]*7] = np.round(N_Wrk_RC2_RS_adj2[i,y:y+rep_phases[j]*7] * max_worker_tot/summ)
        y = y+rep_phases[j]*7
       


In [ ]:
# #elevator components adjustment to ensure the repair time is divided equally across stories 
# elev_array = np.maximum.reduce(N_Wrk_RC2_RS_adj3[:,5:-1:7].T)
# n_wrk_elev = max(RC_table[RC_table["Repair Sequence"]==6]["Workers per damaged Qty"])
# elev_array = np.minimum(elev_array, np.ones((1,len(indx_repairable)))*n_elev*n_wrk_elev)
# indx_elev_array = np.argmax(np.sum((N_Wrk_RC2_RS_adj3[:,5:-1:7]), axis=0))
# N_Wrk_RC2_RS_adj3[:,5+7*indx_elev_array]=elev_array #修正电梯工人数
# if len(np.nonzero(np.sum((N_Wrk_RC2_RS_adj3[:,5:-1:7]), axis=0)))>1:
#     sys.exit("Elevator fragility should be assigned to one floor only")
# RT_RC2_RS_adj4 = np.divide(np.squeeze(RT_RC2_RS), N_Wrk_RC2_RS_adj3, out=np.zeros_like(np.squeeze(RT_RC2_RS)), where=N_Wrk_RC2_RS_adj3!=0) 
# RT_RC2_RS_days = RT_RC2_RS_adj4
# RT_elev_adj = np.maximum.reduce(RT_RC2_RS_days[:,5:-1:7].T)/len(story) #find the max elevator repair time across stories
# RT_RC2_RS_days[:,np.arange(5,len(RT_RC2_RS_days.T),7)] = np.tile(RT_elev_adj,(len(story),1)).T #平均分配到每个楼层

In [ ]:
#elevator components adjustment to ensure the repair time is divided equally across stories 
if len(np.squeeze(np.nonzero(np.sum((N_Wrk_RC2_RS_adj3[:,5:-1:7]), axis=0))))>1:  #补充np.squeeze
    print("Elevator fragility assigned to more than one floor")

elev_story = [1, 30] #电梯所在楼层
elev_story_max = (elev_story[1]-1)*7 #电梯分组 分割线

# elev_array = np.maximum.reduce(N_Wrk_RC2_RS_adj3[:,5:-1:7].T) #沿着数组的指定轴，将数组的元素两两比较，逐步取最大值，最终返回一个包含这些最大值的数组。 5代表RS6; 不包括elev_story_max列
elev_array1 = N_Wrk_RC2_RS_adj3[:,5+(elev_story[0]-1)*7]
elev_array2 = N_Wrk_RC2_RS_adj3[:,5+(elev_story[1]-1)*7]
n_wrk_elev = max(RC_table[RC_table["Repair Sequence"]==6]["Workers per damaged Qty"])
elev_array1 = np.minimum(elev_array1, np.ones((1,len(indx_repairable)))*n_elev[0]*n_wrk_elev)
elev_array2 = np.minimum(elev_array2, np.ones((1,len(indx_repairable)))*n_elev[1]*n_wrk_elev)

# indx_elev_array = np.argmax(np.sum((N_Wrk_RC2_RS_adj3[:,5:elev_story_max:7]), axis=0)) #argmax返回给定数组中最大值的索引
N_Wrk_RC2_RS_adj3[:,5+(elev_story[0]-1)*7]=elev_array1 #修正电梯工人数
N_Wrk_RC2_RS_adj3[:,5+(elev_story[1]-1)*7]=elev_array2 #修正电梯工人数

RT_RC2_RS_adj4 = np.divide(np.squeeze(RT_RC2_RS), N_Wrk_RC2_RS_adj3, out=np.zeros_like(np.squeeze(RT_RC2_RS)), where=N_Wrk_RC2_RS_adj3!=0) #换算成天数
RT_RC2_RS_days = RT_RC2_RS_adj4

RT_elev_adj1 = RT_RC2_RS_days[:,5+(elev_story[0]-1)*7]/(elev_story[1]) #len(story) #find the max elevator repair time across stories
RT_elev_adj2 = RT_RC2_RS_days[:,5+(elev_story[1]-1)*7]/(len(story)-elev_story[1]) #len(story) #find the max elevator repair time across stories

RT_RC2_RS_days[:,np.arange(5,elev_story_max,7)] = np.tile(RT_elev_adj1,((elev_story[1]-1),1)).T #平均分配到每个楼层 #np.tile 用于将数组沿指定的维度进行重复
RT_RC2_RS_days[:,np.arange(elev_story_max+5:len(RT_RC2_RS_days.T):7)] = np.tile(RT_elev_adj2,((len(story)-elev_story[1]+1),1)).T #平均分配到每个楼层 #np.tile 用于将数组沿指定的维度进行重复


In [96]:
RT_RC2_RS_days_mat=np.vstack((header_, (RT_RC2_RS_days)))
pd.DataFrame(np.c_[col.T, RT_RC2_RS_days_mat]).to_csv(os.path.join(output_path,r'RT_RSeq_FR.csv'), header=False, index=False)

In [98]:
#需要保存的变量
# 定义pickle文件的路径
pickle_file_path = os.path.join(subdir_path, 'RT_results.pkl')

output_dict = {
    'RT_RC2_RS_days': RT_RC2_RS_days
}

# 保存数据到pickle文件
with open(pickle_file_path, "wb") as file:
    pickle.dump(output_dict, file)